# Import libraries, set options, connect to DB

In [1]:
# Configuration code for datawrangling
import pandas as pd
import numpy as np
from datetime import datetime
from geocode import geocode
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import Itenerary, Base
engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

# Read in data as pandas data frame, selecting only certain fields

In [2]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time']

In [3]:
df = pd.read_csv('generated_itineraries.csv', usecols = fields)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Wrange field: destinationIDs

In [4]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

# Remove the 2 cases where the string says null
# Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

# Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)

# Wrangle field: numItenerariesReturned

In [5]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [6]:
#Replace all of the NAs in
df.selectedDestination_name.fillna("", inplace = True)

print("Number of rows before departure date: ")
print(len(df))

Number of rows before departure date: 
25869


# Wrangle Field: departureDate

In [7]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))


# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']


print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
25869
number of rows after removal of anamoulous departureDate cases
25861


# Wrangle Field: distinctID

In [8]:
#It turns out distinc_id correpsonds to a user

# Therefore, create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)


vc = df.primary_key.value_counts()
print(vc[vc > 1])


unique_keys = df.primary_key.unique()
#print(len(unique_keys))

#df.head(n = len(df))

16017f5f592536-094b655d7b8ce4-7636321b-4a640-16017f5f5932bd-1512207331      3
156c0427d8411b-089cb3f5e0f3f-1c114a5c-13c680-156c0427d85c5-1472086519       3
68f3348e-32ff-4756-a6b9-fbf722d5bf76-1460184823                             3
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989343                             3
15ae497624281a-06e6a097b99f59-5e4f2b18-ff000-15ae4976243a16-1489889250      3
2b9795d4-912a-4c5b-b628-1ccd366c488f-1466266682                             2
1657f9a517fcd8-0eec0990ac17ad-49183707-13c680-1657f9a5180e51-1535419115     2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989346                             2
16693ed1-fd53-4577-8cbe-5ca19ff74b89-1509430935                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989357                             2
f67face6-b99b-4ddc-8326-3dbe6998c01c-1490958347                             2
6059a95c-9075-49c1-bfce-dc5420759ed3-1469199106                             2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989377                 

# Create a subset of the datle with sample method to test geocode and database entry logic

In [12]:
#Out put the entire database
#df.head(len(df))

len(df)

25861

In [10]:
#Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.sample(5)

# Output this random sample
sampleDf.head(len(sampleDf))    

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,primary_key
13960,2017-07-18,15d57c1df001cc1-064495d38b24d4-8383667-e1000-1...,1,133,Kanaka Creek Cliff Falls,"13514 112 Avenue, Surrey, BC, Canada",1500391002,15d57c1df001cc1-064495d38b24d4-8383667-e1000-1...
22334,2018-06-02,163bcad8502171-0b10d5840a09c1-737356c-1fa400-1...,1,55,Big Cedar and Kennedy Falls,"1160 Pendrell Street, Vancouver, BC, Canada",1527853708,163bcad8502171-0b10d5840a09c1-737356c-1fa400-1...
11405,2017-05-30,15c52d5b26912b-0c2da3dcfc0917-21557c71-38400-1...,1,146,,"Brighouse, Richmond, BC, Canada",1496014176,15c52d5b26912b-0c2da3dcfc0917-21557c71-38400-1...
4701,2016-08-27,156494412fe2a5-0b28631aa27827-37607b02-13c680-...,1,1,,International District/Chinatown Station - Bay...,1472217503,156494412fe2a5-0b28631aa27827-37607b02-13c680-...
15997,2017-08-29,15e2217f7ca496-020410e4e69156-377e0b36-3d10d-1...,1,118,Buntzen Lake,"Coquitlam Central Station, Coquitlam, BC, Canada",1503870420,15e2217f7ca496-020410e4e69156-377e0b36-3d10d-1...


# Loop through the rows in the dataframe, geocode, add entry to database

In [11]:
# Loop through the subsetted pandas data frame
for index, row in df.iterrows():
   

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(Itenerary.distinctKey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # Try to run the geocode function that returns a dictionary of information
        try:
            geocodeInfo = geocode(locationToGeocode)
            # If geocoding works, set valid to tre
            valid = True

        # If the geocode function doesn't work set valid to false
        except:
            valid = False 

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            databaseEntry = Itenerary(distinctKey=row["primary_key"],
                                      numberItinerariesReturned=row["numItinerariesReturned"],
                                      selectedDestination_id=row["selectedDestination_id"],
                                      selectedDestination_name=row["selectedDestination_name"],
                                      startFromLocation=row["startFromLocation"],
                                      departureDate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalCode=geocodeInfo['postalCode'],
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = Itenerary(distinctKey=row["primary_key"],
                                      numberItinerariesReturned=row["numItinerariesReturned"],
                                      selectedDestination_id=row["selectedDestination_id"],
                                      selectedDestination_name=row["selectedDestination_name"],
                                      startFromLocation=row["startFromLocation"],
                                      departureDate=row["departureDate"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside of database, but will edit information to new row")
        # Get the row what I'm going to modift
        rowToModify = session.query(Itenerary).filter_by(Itenerary.distinctKey==testKey).one()
        
        # Pull the lat and lng coordinates out
        lat = rowToModify.lat
        lng = rowToModify.lng
        
        # Match these lat and lng coordinates to a zipCode
        newZip = mapToPoly(lat, lng)
        
        # Match this new zip to the postalCodeMapped Field in database
        rowToModify.postalCodeMapped = newZip
        
        # Add to database
        session.add(rowToModify)
        session.commit()

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

...

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre

Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry already inside of database
Entry alre